In [8]:
import pandas as pd
import string

# Load the files
test_df = pd.read_csv('test_corpus.csv')
output_df = pd.read_csv('similes_output.csv')
annotated_df = pd.read_excel('annotated_corpus.xlsx')

# Function to normalize text
def normalize_text(text):
    if pd.isna(text):
        return text
    # Lowercase the text
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove extra spaces
    text = text.strip()
    return text

# Rename the "sentence" column in test_df to "Simile"
test_df.rename(columns={'sentence': 'Simile'}, inplace=True)

# Apply normalization to the relevant columns
for df in [test_df, output_df, annotated_df]:
    for column in df.columns:
        df[column] = df[column].apply(normalize_text)

# Merge the dataframes on the normalized "Simile" column
merged_df = pd.merge(pd.merge(test_df, output_df, on='Simile', suffixes=('_test', '_output')), 
                     annotated_df, on='Simile', suffixes=('', '_annotated'))

# Function to calculate precision and recall
def calculate_precision_recall(merged_df, column_output, column_annotated):
    true_positive = sum((merged_df[column_output] == merged_df[column_annotated]) & (~merged_df[column_output].isna()))
    false_positive = sum((merged_df[column_output] != merged_df[column_annotated]) & (~merged_df[column_output].isna()))
    false_negative = sum((merged_df[column_output] != merged_df[column_annotated]) & (~merged_df[column_annotated].isna()))
    
    precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0
    recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0
    
    return precision, recall

# Calculate precision and recall for each category
tenor_precision, tenor_recall = calculate_precision_recall(merged_df, 'Tenor', 'Tenor_annotated')
vehicle_precision, vehicle_recall = calculate_precision_recall(merged_df, 'Vehicle', 'Vehicle_annotated')
shared_property_precision, shared_property_recall = calculate_precision_recall(merged_df, 'Shared Property', 'Shared_property')

# Function to calculate precision and recall for simile
def calculate_simile_precision_recall(merged_df):
    true_positive = sum(~merged_df['Simile'].isna())
    false_positive = 0  # Since we are comparing the same rows from different datasets
    false_negative = sum(~merged_df['Simile'].isna()) - true_positive
    
    precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0
    recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0
    
    return precision, recall

# Calculate precision and recall for Simile
simile_precision, simile_recall = calculate_simile_precision_recall(merged_df)

# Compile results into a dictionary
results = {
    'Tenor': {'Precision': tenor_precision, 'Recall': tenor_recall},
    'Vehicle': {'Precision': vehicle_precision, 'Recall': vehicle_recall},
    'Shared Property': {'Precision': shared_property_precision, 'Recall': shared_property_recall},
    'Simile': {'Precision': simile_precision, 'Recall': simile_recall}
}

print(results)


{'Tenor': {'Precision': 0.7578947368421053, 'Recall': 0.7578947368421053}, 'Vehicle': {'Precision': 0.8842105263157894, 'Recall': 0.8842105263157894}, 'Shared Property': {'Precision': 0, 'Recall': 0}, 'Simile': {'Precision': 1.0, 'Recall': 1.0}}
